In [8]:
from torch import nn
from torch_geometric.data import Data
from pathlib import Path
import torch

In [111]:
test_data_path = Path("/Users/fuchur/data/gnn_tracking/data_example") 

In [43]:
from torch_geometric.transforms import LineGraph

In [88]:
del d

In [118]:
class LineGraphBuilder:
    def __init__(self) -> None:
        self._edge_dim_before_modification = None

    @staticmethod
    def append_node_features(data: Data) -> None:
        data.edge_attr = torch.cat(
            [data.edge_attr, data.x[data.edge_index[0]], data.x[data.edge_index[1]]],
             axis=1
        )

    def hyperedge_features(self, line_data: Data) -> None:
        assert line_data.edge_attr is None
        line_data.edge_attr = torch.cat([
            line_data.x[line_data.edge_index[0], :self._edge_dim_before_modification],
            line_data.x[line_data.edge_index[1], :self._edge_dim_before_modification]
        ], axis=1)

    @staticmethod
    def remove_meaningless_info(data: Data) -> None:
        del data.particle_id
        del data.reconstructable
        del data.sector

    def __call__(self, data: Data) -> Data:
        """Node: Modifies data in place"""
        self._edge_dim_before_modification = data.edge_attr.shape[1]
        self.append_node_features(data)
        self.remove_meaningless_info(data)
        LineGraph()(data)
        self.hyperedge_features(data)

    def get_output_dimensions(self, node_dim: int, edge_dim: int) -> tuple[int, int]:
        """
        
        Returns:
            node dim, edge dim for resulting line graph
        """
        return (2*no_dim+edge_dim, 2*edge_dim)

In [112]:
_d = torch.load(test_data_path / "data21873_s20.pt")

In [113]:
_d

Data(x=[1883, 6], edge_index=[2, 129806], edge_attr=[129806, 4], y=[129806], pt=[1883], particle_id=[1883], reconstructable=[1883], sector=[1883], evtid=[1], s=[1])

In [114]:
LineGraphBuilder()(_d)

In [115]:
_d

Data(x=[129806, 16], edge_index=[2, 21398742], y=[129806], pt=[1883], evtid=[1], s=[1], num_nodes=129806, edge_attr=[21398742, 8])

In [ ]:
class KNNLineGraphBuilder(nn.Module):
    def __init__(
            self,
            node_indim: int,
            edge_indiem: int,
            node_outdim: int,

    ):
        self.hnode_encoder = MLP(
            2*node_indim + edge_indim,
        )

    def forward(self, data: Data) -> Data:
        hnode_features = torch.cat([
           data.edge_attr,
           data.node_attr[data.edge_index[0]], 
           data.node_attr[data.edge_index[1]], 
        ], axis=1)
        lg = Data(
            x=hnode_features,
            y=data.y,
            evtid=data.evtid,
            s=data.s,
        )

In [109]:
from gnn_tracking.models.interaction_network import InteractionNetwork as IN
from gnn_tracking.models.mlp import MLP
from gnn_tracking.models.resin import ResIN
from torch import Tensor


class LineGraphKNNEC(nn.Module):
    def __init__(
        self,
        *,
        node_indim: int,
        edge_indim: int,
        interaction_node_dim=5,
        interaction_edge_dim=4,
        hidden_dim=40,
        L_ec=3,
        alpha_ec_node: float = 0.5,
        residual_type="skip1",
        residual_kwargs: dict = None,
    ):
        super().__init__()
        if residual_kwargs is None:
            residual_kwargs = {}

        self.ec_hnode_encoder = MLP(
            hnode_indim, interaction_node_dim, hidden_dim=hidden_dim, L=2, bias=False
        )
        self.ec_resin = ResIN(
            node_dim=interaction_node_dim,
            edge_dim=interaction_edge_dim,
            object_hidden_dim=hidden_dim,
            relational_hidden_dim=hidden_dim,
            alpha_node=alpha_ec_node,
            alpha_edge=0,
            n_layers=L_ec,
            residual_type=residual_type,
            residual_kwargs=residual_kwargs,
        )

        self.W = MLP(input_size=interaction_node_dim, output_size=1, hidden_dim=hidden_dim, L=3)

    def forward(
        self,
        data: Data,
    ) -> Tensor:
        # apply the edge classifier to generate edge weights
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
        h_ec = self.relu(self.ec_node_encoder(x))
        edge_attr_ec = self.relu(self.ec_edge_encoder(edge_attr))
        h_ec, *_ = self.ec_resin(
            h_ec, edge_index, edge_attr_ec
        )
        edge_weights = torch.sigmoid(self.W(w_input))
        return edge_weights